In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
imdb_dir = '/content/gdrive/My Drive/pytest/aclImdb_v1_small/aclImdb/'
import os
train_dir = os.path.join(imdb_dir, 'train') # aclimdb 폴더의 훈련 데이터 내용을 가져온다
labels = [] ; texts = []

## 데이터 로딩

In [4]:
for label_type in ['neg','pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name,fname), encoding='utf-8')
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

## 데이터 확인

In [5]:
print('texts 0:', texts[0])
print('texts len:', len(texts))

print('labels 0:', labels[0])
print('labesls len:', len(labels))

print('texts type:', type(texts))

texts 0: This is it. This is the one. This is the worst movie ever made. Ever. It beats everything. I have never seen worse. Retire the trophy and give it to these people.....there's just no comparison.<br /><br />Even three days after watching this (for some reason I still don't know why) I cannot believe how insanely horrific this movie is/was. Its so bad. So far from anything that could be considered a movie, a story or anything that should have ever been created and brought into our existence.<br /><br />This made me question whether or not humans are truly put on this earth to do good. It made me feel disgusted with ourselves and our progress as a species in this universe. This type of movie sincerely hurts us as a society. We should be ashamed. I really cannot emphasize that our global responsibility as people living here and creating art, is that we need to prevent the creation of these gross distortions of our reality for our own good. It's an embarrassment. I don't know how on

![image](https://user-images.githubusercontent.com/59672592/126786119-10df5512-67ba-4639-8710-88612b826788.png)  
validation set은 학습이 이미 완료된 모델을 검증하기위한 dataset  
train set은 학습과 검증이 완료된 모델의 성능을 평가하기위한 dataset  
Train set은 모델을 학습하기 위한 dataset  
  
validation set과 test set의 공통점은 이 데이터를 통해 모델을 update 즉, 학습을 시키지 않는다

## 데이터 Tokenizing

In [6]:
%tensorflow_version 2.x
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import math

validation_ratio = math.floor(len(texts) * 0.3) # 최적 모델 판정을 위한 검증 샘플은 전체의 30%로 한다
max_words = 10000 # 데이터셋에서 가장 빈도 높은 9,999 개의 단어만 사용한다
maxlen = 200 # 항상 각 문장의 길이가 200 단어가 되도록 고정한다

tokenizer = Tokenizer(num_words=max_words) # 상위빈도 max_words 개의 단어를 추려내는 Tokenizer 객체 생성
tokenizer.fit_on_texts(texts) # texts 내용에 대한 단어 인덱스를 구축한다. 사용할 단어가 결정된다
word_index = tokenizer.word_index # 단어와 인덱스의 쌍을 가져온다

## Tokenizing 결과 확인

In [7]:
print('전체에서 %s개의 고유한 토큰을 찾았습니다.' % len(word_index))
print('word_index type: ', type(word_index))
print('word_index: ', word_index)
# 아직은 max_words(9,999)개가 아닌 전체 88,582개의 단어-빈도 쌍을 보여준다
# texts_to_sequences()를 거쳐야 각 문장에서 빈도순위 max_words(9,999)위까지만 남는다
# word index는 Dictionary 타입이며,
# 토큰으로 선정된 각 단어에 대하여 index가 배당된 것을 알 수 있다

전체에서 2828개의 고유한 토큰을 찾았습니다.
word_index type:  <class 'dict'>
word_index:  {'the': 1, 'and': 2, 'a': 3, 'of': 4, 'to': 5, 'br': 6, 'is': 7, 'in': 8, 'i': 9, 'this': 10, 'it': 11, 'that': 12, 'for': 13, 'film': 14, 'was': 15, 'are': 16, 'with': 17, 'but': 18, 'movie': 19, 'as': 20, 'not': 21, 'you': 22, 'at': 23, 'he': 24, 'on': 25, 'be': 26, 'one': 27, 'have': 28, 'an': 29, 'all': 30, 'his': 31, 'like': 32, 'good': 33, 'has': 34, 'or': 35, 'some': 36, 'just': 37, 'so': 38, 'even': 39, 'my': 40, 'who': 41, 'by': 42, 'story': 43, 'about': 44, 'from': 45, 'time': 46, 'when': 47, 'out': 48, "it's": 49, 'they': 50, 'very': 51, 'no': 52, 'really': 53, 'if': 54, 'acting': 55, 'every': 56, 'him': 57, 'there': 58, 'ever': 59, 'most': 60, 'more': 61, 'people': 62, 'other': 63, 'great': 64, 'how': 65, 'also': 66, 'her': 67, 'bad': 68, 'their': 69, 'your': 70, 'too': 71, 'would': 72, 'made': 73, 'what': 74, 'she': 75, 'stanley': 76, 'me': 77, 'had': 78, 'can': 79, 'characters': 80, 'make': 81, 'plot

## Data Sequencing


In [8]:
# 문자를 숫자로 변환하는 작업을 수행한다
# 각 문장에서 상위 빈도 9,999(max_words)개의 단어만 추출하여 word_index의 숫자를 가지는 리스트로 변환한다.
data = tokenizer.texts_to_sequences(texts) # 빈도 10,000의 Tokenizer 결과가 여기서 반영된다.
print('data 0:', data[0])
print('texts 0:', texts[0]) # texts[0]의 본래 단어들

data 0: [10, 7, 11, 10, 7, 1, 27, 10, 7, 1, 254, 19, 59, 73, 59, 11, 1140, 308, 9, 28, 130, 123, 494, 1141, 1, 1142, 2, 255, 11, 5, 90, 62, 216, 37, 52, 1143, 6, 6, 39, 309, 495, 101, 131, 10, 13, 36, 217, 9, 142, 102, 157, 179, 9, 256, 310, 65, 1144, 686, 10, 19, 7, 15, 106, 38, 68, 38, 180, 45, 384, 12, 124, 26, 687, 3, 19, 3, 43, 35, 384, 12, 107, 28, 59, 115, 1145, 2, 688, 91, 158, 1146, 6, 6, 10, 73, 77, 1147, 1148, 35, 21, 1149, 16, 385, 257, 25, 10, 386, 5, 96, 33, 11, 73, 77, 181, 1150, 17, 1151, 2, 158, 1152, 20, 3, 1153, 8, 10, 1154, 10, 258, 4, 19, 1155, 1156, 311, 20, 3, 259, 108, 107, 26, 1157, 9, 53, 256, 1158, 12, 158, 1159, 1160, 20, 62, 387, 97, 2, 1161, 197, 7, 12, 108, 198, 5, 1162, 1, 1163, 4, 90, 1164, 1165, 4, 158, 388, 13, 158, 389, 33, 49, 29, 1166, 9, 102, 157, 65, 25, 386, 92, 4, 90, 98, 496, 35, 1, 93, 4, 10, 14, 1167, 23, 218, 1168, 12, 50, 78, 3, 182, 8, 260, 497, 9, 102, 157, 74, 258, 4, 689, 1169, 498, 131, 90, 690, 4, 94, 6, 6, 12, 183, 199, 9, 85, 3, 33

## Data Padding 연습

In [9]:
# Padding은 데이터의 길이를 고정시켜 준다
# 지정된 길이에 모자라는 것은 0으로 채우고, 넘치는 것은 잘라낸다
# 넘치는 것을 잘라내는 이유는 문장이 길어질수록 많은 종류의 단어가 나오기 때문이다
# 또한, 길이를 고정해야 텐서의 크기가 맞춰진다
# 기본값으로 단어의 선택은 뒤에서부터 한다
# 또한, nested list를 2D 텐서(2차원 넘파이 배열)로 만든다
from keras.preprocessing.sequence import pad_sequences
sequences = [[1, 2, 3, 4, 5], [1, 2, 3, 4], [1]] # nested list
padded = pad_sequences(sequences, maxlen=3) # 2D tensor
print(padded)
# maxlen = 3 으로 하였으므로, 
# 첫 번째 내부 리스트는 3, 4, 5가 두 번째 내부 리스트는 2, 3, 4가 선택되었다
# 세 번째 내부 리스트는 길이가 모자라므로 앞에서부터 0으로 채웠다

[[3 4 5]
 [2 3 4]
 [0 0 1]]


## Data Padding

In [10]:
data = pad_sequences(data, maxlen=maxlen) 
print('data:', data)
print('data 0:', data[0])
print(len(data[0])) # 200

data: [[ 185  146    6 ...  711  709  266]
 [  24    7   27 ...   86 1231  323]
 [  18   10   19 ...   13    1  394]
 ...
 [2766  145  934 ...    3   13  442]
 [   0    0    0 ...    2 2811  416]
 [   0    0    0 ... 2828    6    6]]
data 0: [ 185  146    6    6   56  694  394  395    7 1176  263  116    2  116
  110    5    3 1177 1178    9   53  130  313    5  103    1 1179  160
  314 1180 1181  395   35  695  696   35 1182  696   35   92 1183  697
   59  110   59   10    7 1184  315 1185    2   37 1186    1  500    6
    6   18    2  264   77   26  698   97    1   60  699  200   44   10
   19    7   12  265    1  161   11  396   12    3  700  316  501    4
  201   15  220  257   91   10 1187    9 1188   86  701    1 1189    4
    1   93   54    9   59  103   57    2 1190   57   91 1191 1192   12
   24 1193  221    2  702  109 1194    4 1195 1196   35    9   86   96
   11   13   57    6    6   54  108   59  703    3  145  704    4 1197
  705   13  158  317 1198  706   37  707 1199  1